In [ ]:
import pandas as pd
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [ ]:
train

,ID,Class
0,377.jpg,MIDDLE
1,17814.jpg,YOUNG
2,21283.jpg,MIDDLE
3,16496.jpg,YOUNG
4,4487.jpg,MIDDLE
...,...,...
19901,2482.jpg,MIDDLE
19902,20085.jpg,YOUNG
19903,19663.jpg,MIDDLE
19904,10132.jpg,MIDDLE


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen=ImageDataGenerator(rescale=1/255,validation_split=0.20)

In [ ]:
train_generator=datagen.flow_from_dataframe(dataframe=train,directory='Train',
                                            x_col="ID",y_col="Class",subset="training",batch_size=32,
                                            shuffle=True,class_mode="categorical",target_size=(100,100))

Found 15925 validated image filenames belonging to 3 classes.


In [ ]:
test_generator=datagen.flow_from_dataframe(dataframe=train,directory='Train',
                                            x_col="ID",y_col="Class",subset='validation',batch_size=32,
                                            shuffle=True,class_mode="categorical",target_size=(100,100))

Found 3981 validated image filenames belonging to 3 classes.


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,Dropout,Conv2D,MaxPool2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50,VGG19
from tensorflow.keras import Model

In [ ]:
Rsnt_model = ResNet50(weights=None, include_top=False, input_shape=(100, 100, 3),classes=3)

In [ ]:
av1 = GlobalAveragePooling2D()(Rsnt_model.output)
fc1 = Dense(256, activation='relu')(av1)
drp1=Dropout(0.35)(fc1)
fc2 = Dense(128, activation='relu')(drp1)
drp2=Dropout(0.4)(fc2)
bat_norm=BatchNormalization()(drp2)
fc3 = Dense(68, activation='relu')(bat_norm)
drp3=Dropout(0.25)(fc3)
fc4 = Dense(34, activation='relu')(drp3)
out = Dense(3, activation='softmax')(fc3)
tl_model = Model(inputs=Rsnt_model.input,outputs=out)
tl_model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 106, 106, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 50, 50, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 50, 50, 64)   256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [ ]:
# compiling the model
tl_model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
# fitting the model
history = tl_model.fit_generator(train_generator,
                         steps_per_epoch = len(train_generator),
                         epochs = 15,
                         validation_data = test_generator,
                         validation_steps = len(test_generator),
                         verbose =1)

Epoch 1/15
498/498 [==============================] - 59s 118ms/step - loss: 1.0097 - accuracy: 0.5128 - val_loss: 0.9710 - val_accuracy: 0.5273
Epoch 2/15
498/498 [==============================] - 57s 115ms/step - loss: 0.9719 - accuracy: 0.5317 - val_loss: 0.9921 - val_accuracy: 0.5315
Epoch 3/15
498/498 [==============================] - 58s 116ms/step - loss: 0.9502 - accuracy: 0.5412 - val_loss: 0.9517 - val_accuracy: 0.5303
Epoch 4/15
498/498 [==============================] - 57s 115ms/step - loss: 0.9033 - accuracy: 0.5828 - val_loss: 0.8756 - val_accuracy: 0.6004
Epoch 5/15
498/498 [==============================] - 58s 116ms/step - loss: 0.8320 - accuracy: 0.6335 - val_loss: 0.8452 - val_accuracy: 0.5948
Epoch 6/15
498/498 [==============================] - 58s 116ms/step - loss: 0.7953 - accuracy: 0.6579 - val_loss: 0.8003 - val_accuracy: 0.6473
Epoch 7/15
498/498 [==============================] - 58s 116ms/step - loss: 0.7688 - accuracy: 0.6667 - val_loss: 0.8404 - val_ac